# LACO END SEM

## Sambeet Kumar | OPD2021006
_____

# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,PowerTransformer, Normalizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

# LOAD DATA

In [2]:
df_concrete = pd.read_excel("C:/Users/sambe/Desktop/sambeet/PGDDSAI/SEM1/Math/assignment/data/Concrete_Data.xlsx")
df_concrete.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


# ALGORITHM

## NMSE

In [3]:
def NMSE(y_test,y_pred):
    return (np.linalg.norm(y_test-y_pred, ord=2)**2)/(np.linalg.norm(y_test, ord=2)**2)

## Steepest Descend

In [4]:
def IHTA(A, b, rigid = True, l=1):

    if rigid == True:
        b = np.concatenate((b, np.zeros(A.shape[1])))
        A = np.concatenate((A,  np.sqrt(l)* np.identity(A.shape[1])))

    m,n = A.shape
    if m!= n:
        b = A.T@b
        A = A.T@A
        
    def pick_large(x, n):
        temp = x.shape[0]
        if n > temp: n = temp
        elif n < 1: n = 1
        to_return = np.sort(x)[::-1]
        return to_return[n-1] 

    x = np.zeros(A.shape[1])
    r = b
    for i in range(100000):
        if i> 0:
            x_old = x
        w = x + np.dot(A.T, r)
        threshold = pick_large(np.abs(w), 4)
        x = np.where(abs(r) >= threshold, w, 0)
        r = b - np.dot(A, x)

        if i> 0:
            if np.linalg.norm(x_old)/np.linalg.norm(x) <  0.0001:
                print(i)
                break
    return x

In [5]:
def k_folds(df,l):
    li =[]
    kf = KFold(n_splits=5)
    df = df.sample(frac=1, random_state=1)
    X = df.iloc[:,:-1] # Features 
    y = df.iloc[:,-1] # Target 
    result= np.array([])
    X = X.values
    y = y.values
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        power = PowerTransformer()
        minmax = MinMaxScaler()
        
        X_train = power.fit_transform(X_train)
        X_test = power.transform(X_test)

        X_train = minmax.fit_transform(X_train)
        X_test = minmax.transform(X_test)

         

        coeff = IHTA(X_train, y_train, rigid = True, l=l)
        y_pred = X_test @ coeff
        # result = np.append(result, mean_absolute_error(y_test, y_pred))
        li.append([mean_squared_error(y_test, y_pred, squared=False), NMSE(y_test,y_pred)])
        li.append([mean_squared_error(y_test, y_pred, squared=False), NMSE(y_test,y_pred)])


    df_result = pd.DataFrame(li, columns = ['RMSE', 'NMSE'])

    return df_result


In [6]:
concrete_results = k_folds(df_concrete, 0.25)
print("Mean",concrete_results.mean())
concrete_results

Mean RMSE    39.509392
NMSE     1.000000
dtype: float64


,RMSE,NMSE
0,40.789851,1.0
1,40.789851,1.0
2,38.540871,1.0
3,38.540871,1.0
4,39.174081,1.0
5,39.174081,1.0
6,38.821130,1.0
7,38.821130,1.0
8,40.221029,1.0
9,40.221029,1.0
